In [1]:
import models_local.resnet_cifar10
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

from utils import get_args
from datasets import IndexedDataset
import logging
import time

args = get_args()

# # Set up logging

logger = logging.getLogger( args.log_dir.split('/')[-1] + time.strftime("-%Y-%m-%d-%H-%M-%S") )

logging.basicConfig(
    filename=f"{args.log_dir}/output.log",
    filemode="a",
    format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    level=logging.INFO,)

# define a Handler which writes INFO messages or higher to the sys.stderr
ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)
logger.addHandler(ch)
args.logger = logger

# Print arguments
args.logger.info("Arguments: {}".format(args))
args.logger.info("Time: {}".format(time.strftime("%Y-%m-%d %H:%M:%S")))

train_dataset = IndexedDataset(args, train=True)
args.train_size = len(train_dataset)

val_loader = torch.utils.data.DataLoader(
        IndexedDataset(args, train=False),
        batch_size=args.batch_size,
        shuffle=False,
        num_workers=args.num_workers,
        pin_memory=True,
    )

model = models_local.resnet_cifar10.resnet50(pretrained=False, progress=True).to(args.device)

Arguments: Namespace(approx_moment=True, approx_with_coreset=True, arch='resnet50', batch_num_mul=5, batch_size=128, check_interval=1, check_thresh_factor=0.1, data_dir='./data', dataset='CIFAR10', device=device(type='cuda'), drop_interval=20, drop_learned=False, drop_thresh=0.1, epochs=10, gamma=0.1, gpu=[0], interval_mul=1.0, log_dir='./logs', logger=<Logger logs-2024-08-09-12-23-00 (INFO)>, lr=0.1, lr_milestones=[100, 150], min_train_size=40000, momentum=0.9, num_classes=10, num_minibatch_coreset=5, num_workers=1, partition_start=0, random_subset_size=0.01, resume_from_epoch=0, runs=1, save_freq=200, seed=0, selection_method='crest', shuffle=True, smtk=0, statedict_dir='./state_dicts', subset_start_epoch=0, train_frac=0.1, use_wandb=False, warm_start_epochs=20, watch_interval=5, weight_decay=0.0001)
Time: 2024-08-09 12:23:00


load dataloader
load dataloader


In [2]:
from trainers import CRESTTrainer
trainer = CRESTTrainer(args, model, train_dataset, val_loader )

In [3]:
trainer.train()

Epoch 0 LR 0.000000
Epoch 0:	Train Loss: 2.762605	Train Acc: 0.101758	Val Loss: 2.516127	Val Acc: 0.105300
Epoch 0:	Data Loading Time: 0.001307	Forward Time: 0.013011	Backward Time: 0.015480
Epoch 1 LR 0.005000
[||||||||||||||||||||]100% [Iteration 13 of 13]Updating train loader and weights with subset of size 650


Epoch [1] [Greedy], pred size: (500, 10)
Greedy: selecting [13 13 13 13 13 13 13 13 13 13] elements
time (sec) for computing facility location: (0.0002856254577636719, 0.00025343894958496094, 0.0002627372741699219, 0.00035643577575683594, 0.00036334991455078125, 0.0002682209014892578, 0.0003025531768798828, 0.0002455711364746094, 0.00024771690368652344, 0.00024628639221191406) similarity time (0.0008320808410644531, 0.00023412704467773438, 0.00024247169494628906, 0.00023698806762695312, 0.00027251243591308594, 0.0002503395080566406, 0.00030684471130371094, 0.00022935867309570312, 0.0002799034118652344, 0.0002200603485107422)
Epoch [1] [Greedy], pred size: (500, 10)
Greedy: selecting [13 13 13 13 13 13 13 13 13 13] elements
time (sec) for computing facility location: (0.0003192424774169922, 0.00030922889709472656, 0.00027251243591308594, 0.00026297569274902344, 0.0002720355987548828, 0.00026917457580566406, 0.0002536773681640625, 0.0002512931823730469, 0.00025653839111328125, 0.00025510

/home/mheuillet/Desktop/robust_training/.venv/lib/python3.8/site-packages/torch/autograd/__init__.py:266: UserWarning: Using backward() with create_graph=True will create a reference cycle between the parameter and its gradient which can cause a memory leak. We recommend using autograd.grad when creating the graph to avoid this. If you have to use this function, make sure to reset the .grad fields of your parameters to None after use to break the cycle and avoid the leak. (Triggered internally at ../torch/csrc/autograd/engine.cpp:1177.)
  Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacity of 2.94 GiB of which 6.12 MiB is free. Including non-PyTorch memory, this process has 2.92 GiB memory in use. Of the allocated memory 2.77 GiB is allocated by PyTorch, and 48.96 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
arg